[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/langgraph/02-ollama-langgraph-agent.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/langgraph/02-ollama-langgraph-agent.ipynb)

# Ollama LangGraph Agent

LangGraph is one of the most powerful frameworks for build AI agents, and Ollama one of the most popular frameworks for running local LLMs. Bringing both together allows us to run agentic workflows at little-to-no cost. In this example we will see how.

In [ ]:
!apt-get install graphviz libgraphviz-dev pkg-config

In [ ]:
!pip install -qU \
    langchain==0.2.12 \
    langchain-core==0.2.29 \
    langgraph==0.2.3 \
    langchain-ollama==0.1.1 \
    pygraphviz==1.12  # for visualizing

## Web Search Tool

In [2]:
from getpass import getpass

api_key = getpass("Enter your API key: ")

Our agent will be able to perform *two* actions, `restaurant_search` and `restaurant_detail`, they will perform the following:

* The `restaurant_search` action will use two sequential API calls; `POST /typeahead` to return a location ID for our chosen city, and `POST /search` to return a list of restaurants for that location.
* The `restaurant_detail` action will use two parallel API calls; `POST /detail` to retrieve more information about a specific restaurant, and `POST /reviews` to retrieve a set of reviews from that restaurant.

Let' see what the API returns to us:

In [3]:
import requests

host = "https://api.search.brave.com/res/v1/web/search"
headers = {
    "Accept": "application/json",
    "Accept-Encoding": "gzip",
    "X-Subscription-Token": api_key
}

res = requests.get(
    host,
    headers=headers,
    params={"q": "restaurant near EUR, roma"}
)
res.json()

{'query': {'original': 'restaurant near EUR, roma',
  'show_strict_warning': False,
  'is_navigational': False,
  'is_geolocal': True,
  'local_decision': 'mix',
  'local_locations_idx': 0,
  'is_news_breaking': False,
  'spellcheck_off': True,
  'country': 'us',
  'bad_results': False,
  'should_fallback': False,
  'postal_code': '',
  'city': '',
  'header_country': '',
  'more_results_available': True,
  'state': ''},
 'mixed': {'type': 'mixed',
  'main': [{'type': 'web', 'index': 0, 'all': False},
   {'type': 'web', 'index': 1, 'all': False},
   {'type': 'web', 'index': 2, 'all': False},
   {'type': 'web', 'index': 3, 'all': False},
   {'type': 'web', 'index': 4, 'all': False},
   {'type': 'web', 'index': 5, 'all': False},
   {'type': 'web', 'index': 6, 'all': False},
   {'type': 'web', 'index': 7, 'all': False},
   {'type': 'web', 'index': 8, 'all': False},
   {'type': 'web', 'index': 9, 'all': False},
   {'type': 'web', 'index': 10, 'all': False},
   {'type': 'web', 'index': 11, 

There's a lot of information here that we simply don't need. We can create a pydantic class that will help us structure the data that we *do want* to keep:

In [4]:
from pydantic import BaseModel

class Page(BaseModel):
    title: str
    url: str
    description: str

    def __str__(self):
        """LLM-friendly string representation of the page."""
        return f"Title: {self.title}\nURL: {self.url}\nDescription: {self.description}"

Then we parse our json output like so:

In [5]:
pages = [Page(**page) for page in res.json()["web"]["results"]]
pages[:3]

[Page(title='Restaurants Near Holiday Inn Rome - Eur Parco Dei Medici', url='https://www.ihg.com/holidayinn/hotels/us/en/rome/romdm/hoteldetail/dining', description='Find the best <strong>restaurants</strong> <strong>near</strong> Holiday Inn Rome - <strong>Eur</strong> Parco Dei Medici, selected by our staff.'),
 Page(title='The 22 Best Restaurants in EUR, Rome | Quandoo', url='https://www.quandoo.it/en/roma-eur', description='Planning to eat out in <strong>EUR</strong>, Rome? Compare 22 different <strong>restaurants</strong> to find the dish you&#x27;re looking for. Reserve a table online now using Quandoo <strong>restaurant</strong> booking system in Rome.'),
 Page(title='10 Migliori Ristoranti EUR (Roma) su Tripadvisor - Leggi ...', url='https://www.tripadvisor.com/Restaurants-g187791-zfn15621866-Rome_Lazio.html', description='<strong>EUR</strong> <strong>Restaurants</strong> - Rome, Lazio: See 14,492 Tripadvisor traveler reviews of 14,492 <strong>restaurants</strong> in Rome <stro

Once we pass these page objects to our LLM, we can use their `__str__` representation to provide a more LLM-friendly format.

In [6]:
print(str(pages[0]))

Title: Restaurants Near Holiday Inn Rome - Eur Parco Dei Medici
URL: https://www.ihg.com/holidayinn/hotels/us/en/rome/romdm/hoteldetail/dining
Description: Find the best <strong>restaurants</strong> <strong>near</strong> Holiday Inn Rome - <strong>Eur</strong> Parco Dei Medici, selected by our staff.


Let's put all of this together into a single `tool` that our LLM will be connected to for function calling.

In [19]:
def web_search(query: str) -> list[Page]:
    """Provides access to web search. You can use this tool to find restaurants.
    Best results can be found by providing as much context as possible, including
    location, cuisine, and the fact that you're looking for a restaurant, cafe,
    etc.
    """
    res = requests.get(
        host,
        headers=headers,
        params={"q": query}
    )
    pages = [Page(**page) for page in res.json()["web"]["results"]]
    return pages

# we invoke the tool like so:
out = web_search(query="best pizza near EUR, roma")
out[:3]

[Page(title='12 Best Pizzerias in Rome Right Now, Picked By A Local', url='https://www.timeout.com/rome/restaurants/best-pizza-in-rome', description='From Romana bases to a Neopolitan crust, here are the <strong>best</strong> <strong>pizza</strong> restaurants in Rome, Italy right now.'),
 Page(title='The 10 Best Pizza in EUR Rome - Tripadvisor', url='https://www.tripadvisor.com/Restaurants-g187791-c31-zfn15621866-Rome_Lazio.html', description='<strong>Best</strong> <strong>Pizza</strong> <strong>in</strong> <strong>EUR</strong> Rome, Lazio: Find Tripadvisor traveller reviews of <strong>EUR</strong> Rome <strong>Pizza</strong> places and search by price, location, and more.'),
 Page(title='THE 10 BEST Pizza Places in Rome (Updated 2024) - Tripadvisor', url='https://www.tripadvisor.com/Restaurants-g187791-c31-Rome_Lazio.html', description='<strong>Best</strong> <strong>Pizza</strong> in Rome, Lazio: Find Tripadvisor traveller reviews of Rome <strong>Pizza</strong> places and search by p

## Web Scraper

Via the Brave Search API, our agent can search the web — *but* it only receives very limited information from the results provided. We have webpage titles, descriptions, and URLs. That information is *not* enough for our agent to provide quality advice on where to find the best pizza. To do this, our agent needs to be able to visit the web pages themselves and scrape *more* information from them. We will create a *web_scraper* tool to make that happen.

We will take a URL from above:

In [8]:
pages[0].url

'https://www.ihg.com/holidayinn/hotels/us/en/rome/romdm/hoteldetail/dining'

In [9]:
requests.get(pages[0].url).content

b'\n<!DOCTYPE HTML>\n<html lang="en-US" dir="ltr">\n\n    <head>\n    <meta charset="UTF-8"/>\n    <title>Restaurants Near Holiday Inn Rome - Eur Parco Dei Medici</title>\n    \n    <meta name="description" content="Find the best restaurants near Holiday Inn Rome - Eur Parco Dei Medici, selected by our staff."/>\n    <meta name="template" content="holiday-inn-dining"/>\n    <meta name="viewport" content="width=device-width, initial-scale=1"/>\n    <script type="text/javascript" src="https://www.ihg.com/assets/ccf6b58295c947908cf624cdd11c465f10016e877cf"  ></script><script>var isPageServedFromAEM = true;</script>\n\t\n\n    \n\n\n \n\n\n\n    <meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta name="viewport" content="width=device-width,initial-scale=1.0"><!--[if lte IE 9]><link rel="stylesheet" href="//www.ihg.com/etc/clientlibs/static/uhf/vendor.2024071702.css"><link rel="stylesheet" href="//www.ihg.com/etc/clientlibs/static/uhf/global.2024071702.

Parse the HTML with beautifulsoup:

In [10]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(requests.get(pages[0].url).content, "html5lib")
soup.find("h1").text

'DINING'

And pull out the elements that would typically contain the text content *we want* from a webpage:

In [11]:
elements = soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p", "span"])

page_text = ""

# merge the text of all elements, using markdown formatting for headers
for element in elements:
    if element.name[0] == "h":
        page_text += f"{'#' * int(element.name[1])}{element.text}\n"
    else:
        page_text += f"{element.text}\n"

print(page_text)


							
							Your session will expire in 5 minutes, 0 seconds, due to inactivity.
5
minutes
0
seconds
You're currently viewing this site in a different language. Would you like this to make your default language? 

                    YES
                    YES
                    NO
                

Costs 13p per minute + phone company's access charge

My stays
Join for free
Sign in
Join for free
Sign in
Sign in
user first name
user first name
· 
user points

				
				Your session has expired. Please  sign in   to your profile
 sign in  

user first name
user points


Book Now
View Prices
See all photos

#DINING
The restaurant La Serra, located on the hotel's ground floor, serves authentic Italian cuisine, ranging from delicious pasta to steak and chips. The restaurant team will suggest the best choice from our wine cellar and DOC wines.
###
            
            

            
            
            
            

            
            
            
              
      

The format isn't perfect, but our LLM will be able to parse this and find the info it needs. Let's wrap all of this into another `tool`:

In [20]:
def scrape_webpage(url: str) -> str:
    """Provides access to web scraping. You can use this tool to scrape a webpage.
    Many webpages may return no information due to JS or adblock issues, if this
    happens, you must use a different URL.
    """
    soup = BeautifulSoup(requests.get(url).content, "html5lib")
    elements = soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p", "span"])
    print(len(elements))
    page_text = ""
    for element in elements:
        if element.name[0] == "h":
            page_text += f"{'#' * int(element.name[1])}{element.text.strip()}\n"
        else:
            page_text += f"{element.text.strip()}\n"
    return page_text

# run again like so:
out = scrape_webpage(url=pages[1].url)
print(out)

818
Looks like something didn't work quite right. Please reload this page.Reload
Looks like something didn't work quite right. Please reload this page.

Near me
For restaurants
Restaurants
Restaurants
Rome
Rome
EUR
EUR
View map
Bookable online
Cuisine
American
(2)
Asian
(2)
Burgers
(2)
Chinese
(1)
Contemporary
(1)
Enoteca
(2)
Fish
(3)
Gourmet
(1)
International
(3)
Italian
(17)
Japanese
(3)
Mediterranean
(1)
Pizza
(8)
Roman
(4)
Steak
(3)
Sushi
(1)
Show all
Review score






1
2
3
4
5
6
Restaurant Price




€
€€
€€€
€€€€
Bookable online

#22 Restaurants in EUR, Rome
Book a table:
1
2
3
4
5
6
7
8
9
10
11
12

August 2024
12:00 am
12:00 am
12:30 am
12:30 am
1:00 am
1:00 am
1:30 am
1:30 am
2:00 am
2:00 am
2:30 am
2:30 am
3:00 am
3:00 am
3:30 am
3:30 am
4:00 am
4:00 am
4:30 am
4:30 am
5:00 am
5:00 am
5:30 am
5:30 am
6:00 am
6:00 am
6:30 am
6:30 am
7:00 am
7:00 am
7:30 am
7:30 am
8:00 am
8:00 am
8:30 am
8:30 am
9:00 am
9:00 am
9:30 am
9:30 am
10:00 am
10:00 am
10:30 am
10:30 am
11:00 am
11:00

### Final Answer "Tool"

Alongside our two tools we will have a final, third tool, called `final_answer`. The final answer tool will be called whenever the LLM has finished pulling info from the other two tools and is ready to provide a *final answer* to the user.

In [21]:
def final_answer(answer: str, phone_number: str, address: str, website: str):
    """Returns a natural language response to the user. There are four sections 
    to be returned to the user, those are:
    - `answer`: the final natural language answer to the user's question.
    - `phone_number`: the phone number of top recommended restaurant.
    - `address`: the address of the top recommended restaurant.
    - `website`: the website of the top recommended restaurant.
    """
    return ""

## Graph Construction

### Agent State

In [22]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

### LLM

The LLM acts as our decision maker and generator of our final output, we will later call this component the `oracle` as our *decision-maker*. For this we are using Ollama and `Llama 3.1`, once initialized we integrate it into a runnable pipeline of our Oracle. The system prompt for our `oracle` will be:

In [39]:
system_prompt = """You are the oracle, the great AI decision maker.
Given the user's query you must decide what to do with it based on the
list of tools provided to you.

Your goal is to provide the user with the best possible restaurant
recommendation. Including key information about why they should consider
visiting or ordering from the restaurant, and how they can do so, ie by
providing restaurant address, phone number, website, etc.

If you see that a tool has been used (in the scratchpad) with a particular
query, do NOT use that same tool with the same query again. You may use the
web scraper tool a few times, but never using the same URL twice.

Once you have collected plenty of information to answer the user's question
(stored in the scratchpad) use the final_answer tool. However, if the user
asks a question or says something unrelated to restaurants, you must use the
final_answer tool directly.

Note, when using a tool, you provide the tool name and the arguments to use
then stop, this will execute the tool without any further input needed from
you. You MUST ONLY use one tool per call."""

Alongside our system prompt, we must also pass Ollama the schema of our functions for tool calls. [Tool calling](https://ollama.com/blog/tool-support) is a relatively new feature in Ollama and is used by providing function schemas to the `tools` parameter when calling our LLM.

We use `FunctionSchema` object with `to_ollama` from `semantic-router` to transform our functions into correctly formatted schemas.

In [40]:
from semantic_router.utils.function_call import FunctionSchema

# create the function calling schema for ollama
web_search_schema = FunctionSchema(web_search).to_ollama()
# TODO deafult None value for description and fix required fields in SR
web_search_schema["function"]["parameters"]["properties"]["query"]["description"] = None
web_search_schema

{'type': 'function',
 'function': {'name': 'web_search',
  'description': "Provides access to web search. You can use this tool to find restaurants.\nBest results can be found by providing as much context as possible, including\nlocation, cuisine, and the fact that you're looking for a restaurant, cafe,\netc.",
  'parameters': {'type': 'object',
   'properties': {'query': {'description': None, 'type': 'string'}},
   'required': []}}}

In [41]:
scrape_webpage_schema = FunctionSchema(scrape_webpage).to_ollama()
# TODO add to SR
scrape_webpage_schema["function"]["parameters"]["properties"]["url"]["description"] = None
scrape_webpage_schema

{'type': 'function',
 'function': {'name': 'scrape_webpage',
  'description': 'Provides access to web scraping. You can use this tool to scrape a webpage.\nMany webpages may return no information due to JS or adblock issues, if this\nhappens, you must use a different URL.',
  'parameters': {'type': 'object',
   'properties': {'url': {'description': None, 'type': 'string'}},
   'required': []}}}

In [45]:
final_answer_schema = FunctionSchema(final_answer).to_ollama()
# TODO add to SR
for key in final_answer_schema["function"]["parameters"]["properties"].keys():
    final_answer_schema["function"]["parameters"]["properties"][key]["description"] = None
final_answer_schema

{'type': 'function',
 'function': {'name': 'final_answer',
  'description': "Returns a natural language response to the user. There are four sections \nto be returned to the user, those are:\n- `answer`: the final natural language answer to the user's question.\n- `phone_number`: the phone number of top recommended restaurant.\n- `address`: the address of the top recommended restaurant.\n- `website`: the website of the top recommended restaurant.",
  'parameters': {'type': 'object',
   'properties': {'answer': {'description': None, 'type': 'string'},
    'phone_number': {'description': None, 'type': 'string'},
    'address': {'description': None, 'type': 'string'},
    'website': {'description': None, 'type': 'string'}},
   'required': []}}}

Now we can test our LLM!

---

**❗️ Make sure you have Ollama running locally and you have already downloaded the model with `ollama pull llama3-groq-tool-use:8b`!**

```

In [46]:
import ollama

res = ollama.chat(
    model="llama3.1:8b",
    messages=[
        {"role": "system", "content": system_prompt},
        # chat history will go here
        {"role": "user", "content": "hello there"}
        # scratchpad will go here
    ],
    tools=[scrape_webpage_schema, web_search_schema, final_answer_schema]
)

In [47]:
res

{'model': 'llama3.1:8b',
 'created_at': '2024-08-15T08:49:38.515049Z',
 'message': {'role': 'assistant',
  'content': '',
  'tool_calls': [{'function': {'name': 'final_answer',
     'arguments': {'address': '',
      'answer': 'Hello! How can I assist you today?',
      'phone_number': '',
      'website': ''}}}]},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 1552482833,
 'load_duration': 38047875,
 'prompt_eval_count': 656,
 'prompt_eval_duration': 643721000,
 'eval_count': 38,
 'eval_duration': 867421000}

We can see here that the LLM is correctly deciding to use the `final_answer` tool to respond to the user. Let's see if we can get it to use the web search tool.

In [48]:
res = ollama.chat(
    model="llama3.1:8b",
    messages=[
        {"role": "system", "content": system_prompt},
        # chat history will go here
        {"role": "user", "content": "hi, I'm looking for the best pizzeria in EUR, rome"}
        # scratchpad will go here
    ],
    tools=[scrape_webpage_schema, web_search_schema, final_answer_schema]
)
res

{'model': 'llama3.1:8b',
 'created_at': '2024-08-15T09:27:55.343173Z',
 'message': {'role': 'assistant',
  'content': '',
  'tool_calls': [{'function': {'name': 'web_search',
     'arguments': {'query': 'best pizzeria in EUR, rome'}}}]},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 4985783792,
 'load_duration': 2562037708,
 'prompt_eval_count': 670,
 'prompt_eval_duration': 1859329000,
 'eval_count': 25,
 'eval_duration': 559697000}

That looks great! Now we just need to wrap this with the ability to contain chat history and the agent scratchpad — before adding everything into our graph.

In [17]:
from langchain_core.messages import ToolCall, ToolMessage
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0,
)

tools=[
    scrape_webpage,
    web_search,
    final_answer
]

# define a function to transform intermediate_steps from list
# of AgentAction to scratchpad string
def create_scratchpad(intermediate_steps: list[AgentAction]):
    research_steps = []
    for i, action in enumerate(intermediate_steps):
        if action.log != "TBD":
            # this was the ToolExecution
            research_steps.append(
                f"Tool: {action.tool}, input: {action.tool_input}\n"
                f"Output: {action.log}"
            )
    return "\n---\n".join(research_steps)

oracle = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "scratchpad": lambda x: create_scratchpad(
            intermediate_steps=x["intermediate_steps"]
        ),
    }
    | prompt
    | llm.bind_tools(
        tools=tools,
        #function_call={"name": "final_answer"}
    )
)

Test our `oracle` to confirm it works:

In [18]:
from semantic_router.utils.function_call import FunctionSchema

schema = FunctionSchema(scrape_webpage).to_ollama()
schema

/Users/jamesbriggs/opt/anaconda3/envs/ollama-langgraph/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
/Users/jamesbriggs/opt/anaconda3/envs/ollama-langgraph/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'type': 'function',
 'function': {'name': 'scrape_webpage',
  'description': 'Provides access to web scraping. You can use this tool to scrape a webpage.\nMany webpages may return no information due to JS or adblock issues, if this\nhappens, you must use a different URL.',
  'parameters': {'type': 'object',
   'properties': {'url': {'description': FieldInfo(annotation=NoneType, required=False, default=None, description='The description of the parameter'),
     'type': 'string'}},
   'required': []}}}

In [ ]:
scrape_webpage.args_schema.schema()

In [ ]:
inputs = {
    "input": "hello there",
    "chat_history": [],
    "intermediate_steps": [],
}
out = oracle.invoke(inputs)
out

In [ ]:
import ollama

res = ollama.chat(
    model="llama3.1:8b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "hello there"}
    ]
)

In [ ]:
res

In [ ]:
out.tool_calls

We can see the tool was used:

In [ ]:
out.content

### Graph Nodes

We have defined the different logical components of our graph, but we need to execute them in a langgraph-friendly manner — for that they must consume our `AgentState` and return modifications to that state. We will do this for all of our components via three functions:

* `run_oracle` will handle running our oracle LLM.
* `router` will handle the *routing* between our oracle and tools.
* `run_tool` will handle running our tool functions.

In [ ]:
def run_oracle(state: list):
    print("run_oracle")
    print(f"intermediate_steps: {state['intermediate_steps']}")
    out = oracle.invoke(state)
    tool_name = out.tool_calls[0]["name"]
    tool_args = out.tool_calls[0]["args"]
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log="TBD"
    )
    return {
        "intermediate_steps": [action_out]
    }

def router(state: list):
    # return the tool name to use
    if isinstance(state["intermediate_steps"], list):
        return state["intermediate_steps"][-1].tool
    else:
        # if we output bad format go to final answer
        print("Router invalid format")
        return "final_answer"

# we use this to map tool names to tool functions
tool_str_to_func = {
    "scrape_webpage": scrape_webpage,
    "web_search": web_search,
    "final_answer": final_answer
}

def run_tool(state: list):
    # use this as helper function so we repeat less code
    tool_name = state["intermediate_steps"][-1].tool
    tool_args = state["intermediate_steps"][-1].tool_input
    print(f"{tool_name}.invoke(input={tool_args})")
    # run tool
    out = tool_str_to_func[tool_name].invoke(input=tool_args)
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log=str(out)
    )
    return {"intermediate_steps": [action_out]}